<a href="https://colab.research.google.com/github/Alexandr20i/Th_coding/blob/main/Th_coding_LR2_Belov_Vinogradov_Grigorev_Semenickhin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Лабораторная работа №2

In [ ]:
import  numpy as np

# Часть 1

## 2.1 Сформировать порождающую матрицу линейного кода (7, 4, 3)

In [ ]:
n, k, d = 7, 4, 3
# Создание единичной матрицы I_k размером 4x4
I_k = np.eye(k, dtype=int)
# Инициализация пустого списка для хранения строк X
X = []

# Генерация всех возможных строк длины 3 с двумя и более единицами
for i in range(2 ** d):
    # Преобразуем число в двоичный вид и заполняем строку
    row = [(i >> j) & 1 for j in range(3)]
    # Проверяем, что в строке не менее двух единиц
    if sum(row) >= 2:
        X.append(row)

# Удаляем дубликаты
X = np.unique(X, axis=0)
# Выбираем первые 4 уникальные строки
X = X[:4]
# Формирование порождающей матрицы G
G = np.hstack((I_k, X))
print(f'порождающая матрица линейного кода (7, 4, 3):\n{G}')

порождающая матрица линейного кода (7, 4, 3):
[[1 0 0 0 0 1 1]
 [0 1 0 0 1 0 1]
 [0 0 1 0 1 1 0]
 [0 0 0 1 1 1 1]]


## 2.2 Сформировать проверочную матрицу на основе порождающей

In [ ]:
n_k = n - k  # Количество проверочных бит (3)

# Создание единичной матрицы I_{n-k} размером 3x3
I_n_k = np.eye(n_k, dtype=int)

# Формирование проверочной матрицы H
H = np.vstack((X, I_n_k))
print(f'Проверочная матрица H:\n{H}')

Проверочная матрица H:
[[0 1 1]
 [1 0 1]
 [1 1 0]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]


## 2.3 Сформировать таблицу синдромов для всех однократных ошибок

In [ ]:
def calculate_syndrome(error_vector,H):
    #Вычисляет синдром для данного вектора ошибки и матрицы H.
    syndrome = (error_vector @ H) % 2
    return tuple(syndrome)

def create_syndrome_table(H):
    #Создает таблицу синдромов для заданной проверочной матрицы.
    num_rows = H.shape[0]
    syndrome_dict = {}

    # Проходим по всем возможным позициям ошибок
    for i in range(num_rows):
        error_vector = np.zeros(num_rows, dtype=int) # Создаем вектор ошибки с нулями
        error_vector[i] = 1  # Вносим ошибку в i-ю позицию
        syndrome = calculate_syndrome(error_vector, H) # Вычисляем синдром
        syndrome_dict[syndrome] = tuple(error_vector) # Сохраняем синдром и соответствующий вектор ошибки в словаре

    return syndrome_dict

syndrome_table = create_syndrome_table(H)
print("Таблица синдромов для однократных ошибок:")
print("Синдром\t\tВектор ошибки")
for syndrome in syndrome_table:
    print(f"{syndrome}\t{syndrome_table[syndrome]}")

Таблица синдромов для однократных ошибок:
Синдром		Вектор ошибки
(0, 1, 1)	(1, 0, 0, 0, 0, 0, 0)
(1, 0, 1)	(0, 1, 0, 0, 0, 0, 0)
(1, 1, 0)	(0, 0, 1, 0, 0, 0, 0)
(1, 1, 1)	(0, 0, 0, 1, 0, 0, 0)
(1, 0, 0)	(0, 0, 0, 0, 1, 0, 0)
(0, 1, 0)	(0, 0, 0, 0, 0, 1, 0)
(0, 0, 1)	(0, 0, 0, 0, 0, 0, 1)


## 2.4 Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова


In [ ]:
def fix_errors(parity_check_matrix, received_word, syndrome_lookup):
    # Исправляет ошибки в принятом кодовом слове с помощью таблицы синдромов.
    # Вычисляем синдром для принятого слова
    current_syndrome = tuple(np.dot(parity_check_matrix.T, received_word) % 2)

    # Проверяем, есть ли синдром в таблице
    if current_syndrome in syndrome_lookup:
        error_pattern = np.array(syndrome_lookup[current_syndrome], dtype=int)  # Получаем вектор ошибок
        corrected_word = (received_word + error_pattern) % 2  # Исправляем слово
        return corrected_word, error_pattern  # Возвращаем исправленное слово и вектор ошибок

    # Если синдром не найден, проверяем пары ошибок
    num_bits = len(received_word)
    for i in range(num_bits):
        for j in range(i + 1, num_bits):  # Избегаем повторяющихся пар
            # Создаем шаблон ошибки для текущей пары индексов
            error_pattern = np.zeros(num_bits, dtype=int)
            error_pattern[i] = 1  # Устанавливаем ошибку в i-ю позицию
            error_pattern[j] = 1  # Устанавливаем ошибку в j-ю позицию

            # Вычисляем новое слово и его синдром
            modified_word = (received_word + error_pattern) % 2
            new_syndrome = tuple(np.dot(parity_check_matrix.T, modified_word) % 2)

            # Проверяем новый синдром
            if new_syndrome in syndrome_lookup:
                corrected_word = (received_word + error_pattern) % 2  # Исправляем слово
                return corrected_word, error_pattern  # Возвращаем исправленное слово и вектор ошибок

    # Если ошибки не обнаружены, возвращаем исходное слово
    return received_word, None  # Возвращаем исходное слово и None для вектора ошибок

In [ ]:
original_msg = np.random.randint(0, 2, size=k)
print(f"Оригинальное сообщение: {original_msg}")

encoded_word =  np.dot(original_msg, G) % 2
print(f"Кодированное сообщение: {encoded_word}")
# внесем однократную ошибку
pos = np.random.randint(len(encoded_word))
corrupted_word = np.array([bit ^ (i == pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = fix_errors(H, corrupted_word, syndrome_table)

if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")

print(f"Исправленное сообщение: {corrected_word}")

print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [1 1 1 1]
Кодированное сообщение: [1 1 1 1 1 1 1]
Сообщение с ошибкой в позиции 6: [1 1 1 1 1 1 0]
Обнаруженная ошибка: [0 0 0 0 0 0 1]
Исправленное сообщение: [1 1 1 1 1 1 1]
Проверка на равенство отправленного слова и полученного: True


## 2.5. Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного.

In [ ]:
print(f"Оригинальное сообщение: {original_msg}")
print(f"Кодированное сообщение: {encoded_word}")
# внесем двукратную ошибку
pos = np.random.choice(len(encoded_word), size=2, replace=False)
corrupted_word = np.array([bit ^ (i in pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = fix_errors(H, corrupted_word, syndrome_table)

if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")

print(f"Исправленное сообщение: {corrected_word}")

print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [1 1 1 1]
Кодированное сообщение: [1 1 1 1 1 1 1]
Сообщение с ошибкой в позиции [2 3]: [1 1 0 0 1 1 1]
Обнаруженная ошибка: [0 0 0 0 0 0 1]
Исправленное сообщение: [1 1 0 0 1 1 0]
Проверка на равенство отправленного слова и полученного: False


# Часть 2

## 2.6. Сформировать порождающую матрицу линейного кода (n, k, 5).

In [ ]:
# Параметры кода
n = 9  # длина кодового слова
k = 4  # длина информационного слова
n_k = n - k
# Примерная матрица P
X = np.array([[1, 1, 1, 1, 0],
              [1, 1, 0, 0, 1],
              [1, 0, 1, 0, 1],
              [0, 1, 1, 0, 1]])
I_k = np.eye(k, dtype=int)
G = np.hstack((I_k, X))
print("Порождающая матрица G:\n", G)

Порождающая матрица G:
 [[1 0 0 0 1 1 1 1 0]
 [0 1 0 0 1 1 0 0 1]
 [0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 1 0 1]]


## 2.7 Сформировать проверочную матрицу на основе порождающей.

In [ ]:
n_k = n - k  # Количество проверочных бит (3)

# Создание единичной матрицы I_{n-k} размером 3x3
I_n_k = np.eye(n_k, dtype=int)

# Формирование проверочной матрицы H
H = np.vstack((X, I_n_k))
print(f'Проверочная матрица H:\n{H}')

Проверочная матрица H:
[[1 1 1 1 0]
 [1 1 0 0 1]
 [1 0 1 0 1]
 [0 1 1 0 1]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]


## 2.8 Сформировать таблицу синдромов для всех однократных и двукратных ошибок.

In [ ]:
def build_syndrome_table(H):
    n = H.shape[1]  # Количество столбцов (размер кодового слова)

    syndromes = {}

    # Однократные ошибки
    for i in range(n):
        error_vec = np.zeros(n, dtype=int)
        error_vec[i] = 1
        S = calculate_syndrome(error_vec, H.T)
        syndromes[S] = tuple(error_vec)

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            error_vec = np.zeros(n, dtype=int)
            error_vec[i] = 1
            error_vec[j] = 1
            S = calculate_syndrome(error_vec, H.T)
            syndromes[S] = tuple(error_vec)

    return syndromes

print("Таблица синдромов для однократных и двухкратных ошибок:")
syndrome_table = build_syndrome_table(H.T)
print("Синдром\t\t\tВектор ошибки")
for syndrome in syndrome_table:
    print(f"{syndrome}\t\t{syndrome_table[syndrome]}")


Таблица синдромов для однократных и двухкратных ошибок:
Синдром			Вектор ошибки
(1, 1, 1, 1, 0)		(1, 0, 0, 0, 0, 0, 0, 0, 0)
(1, 1, 0, 0, 1)		(0, 1, 0, 0, 0, 0, 0, 0, 0)
(1, 0, 1, 0, 1)		(0, 0, 1, 0, 0, 0, 0, 0, 0)
(0, 1, 1, 0, 1)		(0, 0, 0, 1, 0, 0, 0, 0, 0)
(1, 0, 0, 0, 0)		(0, 0, 0, 0, 1, 0, 0, 0, 0)
(0, 1, 0, 0, 0)		(0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 1, 0, 0)		(0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 1, 0)		(0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 1)		(0, 0, 0, 0, 0, 0, 0, 0, 1)
(0, 0, 1, 1, 1)		(1, 1, 0, 0, 0, 0, 0, 0, 0)
(0, 1, 0, 1, 1)		(1, 0, 1, 0, 0, 0, 0, 0, 0)
(1, 0, 0, 1, 1)		(1, 0, 0, 1, 0, 0, 0, 0, 0)
(0, 1, 1, 1, 0)		(1, 0, 0, 0, 1, 0, 0, 0, 0)
(1, 0, 1, 1, 0)		(1, 0, 0, 0, 0, 1, 0, 0, 0)
(1, 1, 0, 1, 0)		(1, 0, 0, 0, 0, 0, 1, 0, 0)
(1, 1, 1, 0, 0)		(1, 0, 0, 0, 0, 0, 0, 1, 0)
(1, 1, 1, 1, 1)		(1, 0, 0, 0, 0, 0, 0, 0, 1)
(0, 1, 1, 0, 0)		(0, 0, 0, 0, 0, 1, 1, 0, 0)
(1, 0, 1, 0, 0)		(0, 0, 0, 0, 1, 0, 1, 0, 0)
(0, 1, 0, 0, 1)		(0, 0, 0, 0, 0, 1, 0, 0, 1)
(1, 0, 0, 0, 1)		(0,

## 2.9. Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [ ]:
original_msg = np.random.randint(0, 2, size=k)
print(f"Оригинальное сообщение: {original_msg}")

encoded_word =  np.dot(original_msg, G) % 2
print(f"Кодированное сообщение: {encoded_word}")
# внесем однократную ошибку
pos = np.random.randint(len(encoded_word))
corrupted_word = np.array([bit ^ (i == pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = fix_errors(H, corrupted_word, syndrome_table)

if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")

print(f"Исправленное сообщение: {corrected_word}")

print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [1 0 1 0]
Кодированное сообщение: [1 0 1 0 0 1 0 1 1]
Сообщение с ошибкой в позиции 3: [1 0 1 1 0 1 0 1 1]
Обнаруженная ошибка: [0 0 0 1 0 0 0 0 0]
Исправленное сообщение: [1 0 1 0 0 1 0 1 1]
Проверка на равенство отправленного слова и полученного: True


## 2.10. Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [ ]:
print(f"Оригинальное сообщение: {original_msg}")
print(f"Кодированное сообщение: {encoded_word}")
# внесем двукратную ошибку
pos = np.random.choice(len(encoded_word), size=2, replace=False)
corrupted_word = np.array([bit ^ (i in pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = fix_errors(H, corrupted_word, syndrome_table)

if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")

print(f"Исправленное сообщение: {corrected_word}")

print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [1 0 1 0]
Кодированное сообщение: [1 0 1 0 0 1 0 1 1]
Сообщение с ошибкой в позиции [6 0]: [0 0 1 0 0 1 1 1 1]
Обнаруженная ошибка: [1 0 0 0 0 0 1 0 0]
Исправленное сообщение: [1 0 1 0 0 1 0 1 1]
Проверка на равенство отправленного слова и полученного: True


## 2.11. Сформировать кодовое слово длины n из слова длины k. Внести трёхкратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного.

In [ ]:
print(f"Оригинальное сообщение: {original_msg}")
print(f"Кодированное сообщение: {encoded_word}")
# внесем трехкратную ошибку
pos = np.random.choice(len(encoded_word), size=3, replace=False)
corrupted_word = np.array([bit ^ (i in pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = fix_errors(H, corrupted_word, syndrome_table)

if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")

print(f"Исправленное сообщение: {corrected_word}")

print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [1 0 1 0]
Кодированное сообщение: [1 0 1 0 0 1 0 1 1]
Сообщение с ошибкой в позиции [6 5 0]: [0 0 1 0 0 0 1 1 1]
Обнаруженная ошибка: [0 0 0 0 1 0 0 1 0]
Исправленное сообщение: [0 0 1 0 1 0 1 0 1]
Проверка на равенство отправленного слова и полученного: False
